In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -q git+https://github.com/samoturk/mol2vec

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install -q rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 843.1 kB/s eta 0:00:00


In [ ]:
from rdkit import Chem

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/METSA/train.csv')

In [ ]:
df.head(3)




,id,polymer,fp_mqns_1,fp_mqns_2,fp_mqns_3,fp_mqns_4,fp_mqns_5,fp_mqns_6,fp_mqns_7,fp_mqns_8,...,fp_o_desc_chi1n,fp_o_desc_chi1v,fp_o_desc_chi2n,fp_o_desc_chi2v,fp_o_desc_chi3n,fp_o_desc_chi3v,fp_o_desc_chi4n,fp_o_desc_chi4v,fp_o_desc_HallKierAlpha,band_gap
0,789,[*]C(=O)c1ccc2c(c1)C(=O)N(c1ccc(C3(c4ccc(N5C(=O)c6ccc([*])cc6C5=O)cc4)c4ccccc4-c4ccccc43)cc1)C2=O,1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.377963,0.377963,0.300679,0.300679,0.245471,0.245471,0.192698,0.192698,0.160238,2.3311
1,2983,[*]CCCCCOC(=O)c1ccc(C(=O)NCCCCCCNC(=O)c2ccc(C(=O)O[*])cc2)cc1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.074074,...,0.449784,0.449784,0.310747,0.310747,0.209719,0.209719,0.132236,0.132236,0.136296,4.3426
2,1037,[*]CNON[*],1,0.0,0.0,0.0,0.0,0.0,0.0,2.000000,...,1.093414,1.093414,0.536822,0.536822,0.266744,0.266744,0.136920,0.136920,0.120000,5.3302


In [ ]:
df['mol'] = df['polymer'].apply(lambda x: Chem.MolFromSmiles(x))

In [ ]:
!pip install -q karateclub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 43.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-common 1.0.0 requires pandas<2.2.0,>=2.0.0, but you have pandas 1.3.5 which is incompatible.
autogluon-core 1.0.0 requires networkx<4,>=3.0, but you have networkx 2.6.3 which is incompatible.
autogluon-core 1.0.0

In [ ]:
from rdkit import Chem
import networkx as nx
from karateclub import Graph2Vec

AttributeError: partially initialized module 'smart_open' has no attribute 'local_file' (most likely due to a circular import)

In [ ]:
def mol_to_nx(mol):
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(),
                   atomic_num=atom.GetAtomicNum(),
                   is_aromatic=atom.GetIsAromatic(),
                   atom_symbol=atom.GetSymbol())

    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType())

    return G

In [ ]:
df['graph'] = df['mol'].apply(lambda x: mol_to_nx(x))

model = Graph2Vec()
model.fit(df['graph'])
hiv_graph2vec = model.get_embedding()

hiv_graph2vec = pd.DataFrame(hiv_graph2vec)
print(hiv_graph2vec)

In [ ]:
df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))
df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())

In [ ]:
from rdkit.Chem import Descriptors
df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [ ]:
df = pd.concat([df, hiv_graph2vec], axis=1)

In [ ]:
df.head(2)

In [ ]:
df.drop(['polymer', 'mol', 'sentence', 'graph'], axis = 1, inplace=True)

In [ ]:
df

,fp_mqns_1,fp_mqns_2,fp_mqns_3,fp_mqns_4,fp_mqns_5,fp_mqns_6,fp_mqns_7,fp_mqns_8,fp_mqns_9,fp_mqns_10,...,fp_o_desc_chi4n,fp_o_desc_chi4v,fp_o_desc_HallKierAlpha,band_gap,num_of_atoms,num_of_heavy_atoms,tpsa,mol_w,num_valence_electrons,num_heteroatoms
0,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.047619,0.119048,...,0.192698,0.192698,0.160238,2.3311,73,49,91.83,634.152872,230,9
1,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.074074,0.000000,0.222222,...,0.132236,0.132236,0.136296,4.3426,69,35,110.80,480.226037,186,10
2,1,0.0,0.0,0.0,0.0,0.000000,0.0,2.000000,0.000000,1.000000,...,0.136920,0.136920,0.120000,5.3302,10,4,33.29,60.032363,24,5
3,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.181818,...,0.136631,0.136631,0.119091,4.5699,52,26,52.60,352.167459,136,6
4,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.333333,...,0.269098,0.269098,0.013333,6.0531,20,8,29.46,114.068080,46,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,1,0.0,0.0,0.0,0.0,0.023810,0.0,0.047619,0.047619,0.190476,...,0.161359,0.191382,0.178810,2.4779,81,55,167.10,744.131485,266,15
2530,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.142857,0.000000,0.357143,...,0.132861,0.132861,0.107143,6.1723,49,21,85.89,302.184172,122,9
2531,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.143097,0.143097,0.032500,4.9643,24,8,0.00,110.109550,46,2
2532,1,0.0,0.0,0.0,0.0,0.153846,0.0,0.000000,0.153846,0.000000,...,0.149962,0.276198,0.116923,3.8894,27,17,25.78,256.012890,82,6


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
tr = pd.read_csv('/content/drive/MyDrive/METSA/trf.csv')

In [ ]:
tr

,fp_mqns_13,fp_mqns_17,fp_mqns_19,fp_mqns_27,fp_mqns_30,fp_mqns_31,fp_smrVsa_5,fp_smrVsa_7,fp_slogPVsa_6,fp_o_desc_nRotBon,fp_o_desc_nRing,fp_o_desc_nAroRing,fp_o_desc_k2,fp_o_desc_k3,fp_o_desc_chi1n,band_gap
0,0.139881,0.428571,0.139881,0.002976,0.526786,0.473214,0.128928,4.503749,3.195676,0.139881,0.214286,0.142857,0.252483,0.094565,0.377963,2.3311
1,0.773148,0.222222,0.620370,0.546296,0.296296,0.148148,1.696712,2.621657,1.797442,0.620370,0.074074,0.074074,0.601248,0.394968,0.449784,4.3426
2,3.875000,0.000000,3.625000,3.750000,0.000000,0.000000,0.000000,0.000000,0.000000,3.625000,0.000000,0.000000,3.755000,3.630000,1.093414,5.3302
3,0.630682,0.272727,0.539773,0.454545,0.363636,0.181818,2.334844,2.205952,2.205952,0.539773,0.090909,0.090909,0.519141,0.352058,0.423286,4.5699
4,0.479167,0.000000,0.291667,0.145833,0.687500,0.145833,6.268995,0.000000,0.000000,0.291667,0.166667,0.000000,0.416579,0.282672,0.530491,6.0531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,0.258929,0.428571,0.214286,0.050595,0.526786,0.473214,0.233118,3.973896,3.428794,0.214286,0.190476,0.142857,0.325058,0.148617,0.400195,2.4779
2530,1.348214,0.000000,0.830357,0.839286,0.000000,0.000000,2.862812,0.000000,0.000000,0.830357,0.000000,0.000000,0.804364,0.821023,0.518083,6.1723
2531,0.859375,0.000000,0.593750,0.468750,0.000000,0.000000,3.343998,1.526863,1.526863,0.593750,0.000000,0.000000,0.592941,0.401980,0.436084,4.9643
2532,0.298077,0.615385,0.288462,0.067308,0.778846,0.451923,0.498745,3.469516,2.747262,0.288462,0.230769,0.230769,0.372913,0.187769,0.412484,3.8894


In [ ]:
!pip install -q autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
predictor = TabularPredictor(label = 'band_gap').fit(tr)

No path specified. Models will be saved in: "AutogluonModels/ag-20240227_030308"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240227_030308"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
C

[1000]	valid_set's rmse: 0.735264


	-0.7331	 = Validation score   (-root_mean_squared_error)
	1.41s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: LightGBM ...
	-0.737	 = Validation score   (-root_mean_squared_error)
	2.31s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.7694	 = Validation score   (-root_mean_squared_error)
	7.24s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	-0.7051	 = Validation score   (-root_mean_squared_error)
	32.49s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.7533	 = Validation score   (-root_mean_squared_error)
	2.08s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.688	 = Validation score   (-root_mean_squared_error)
	4.75s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-0.7564	 = Validation score   (-root_mean_squared_error)
	3.3s	 = Training   runtime
	0.01s	 = Validation runtime
Fitti

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/METSA/tef.csv')

In [ ]:
# test['mol'] = test['polymer'].apply(lambda x: Chem.MolFromSmiles(x))

In [ ]:
# test['mol'] = test['mol'].apply(lambda x: Chem.AddHs(x))
# test['num_of_atoms'] = test['mol'].apply(lambda x: x.GetNumAtoms())
# test['num_of_heavy_atoms'] = test['mol'].apply(lambda x: x.GetNumHeavyAtoms())

In [ ]:
# test['tpsa'] = test['mol'].apply(lambda x: Descriptors.TPSA(x))
# test['mol_w'] = test['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
# test['num_valence_electrons'] = test['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
# test['num_heteroatoms'] = test['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

In [ ]:
# test['aromatic'] = test['polymer'].apply(lambda x: Descriptors.RingCount(Chem.MolFromSmiles(x))>0 )
# test['aromatic1'] = test['polymer'].apply(lambda x: Descriptors.NumAromaticRings(Chem.MolFromSmiles(x))>0)

In [ ]:
# test.drop(['id', 'polymer', 'mol'], axis=1, inplace=True)

In [ ]:
predicted = predictor.predict(test)

In [ ]:
sample_sub = pd.read_csv('/content/drive/MyDrive/METSA/sample_submission.csv')

In [ ]:
sample_sub['band_gap'] = predicted.values

In [ ]:
sample_sub.to_csv('predictions_69.csv', index=False)